In [5]:
import requests
import zipfile
from io import BytesIO

url19 = 'https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/8c233bc2-1879-44ff-a0e4-9b69a9032c54/resource/91006b8a-f018-4627-9353-4dfdab71861d/download/parking-tickets-2019.zip'
url18 = 'https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/8c233bc2-1879-44ff-a0e4-9b69a9032c54/resource/590e5f97-461b-4f72-b448-ef52bbfd1296/download/parking-tickets-2018.zip'
url17 = 'https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/8c233bc2-1879-44ff-a0e4-9b69a9032c54/resource/f1e9e1a3-cc95-4ce4-9dbe-081d9e6edc4a/download/parking-tickets-2017.zip'
url16 = 'https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/8c233bc2-1879-44ff-a0e4-9b69a9032c54/resource/2fbcff61-4c3b-42a7-bdc5-02fb07e728aa/download/parking-tickets-2016.zip'
url20 = ('https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/8c233bc2-1879-44ff-a0e4-9b69a9032c54/resource/0d26a209-6e61-4154-9d70-8a6ad0e2d14d/download/parking-tickets-2020.zip')

url_list = [url16, url17, url18, url19,url20]

def getFiles(url):
    rq = requests.get(url)
    zipfile = zipfile.ZipFile(BytesIO(rq.content))
    return zipfile.extractall('./')

for i in url_list:
    getFiles(i)


In [ ]:
import pandas as pd
import glob
import os

files = os.path.join('./', 'Parking_Tags_Data_*.csv')

files = glob.glob(files)

df = pd.concat(map(pd.read_csv, files), ignore_index=True)


## Province with most tickets between 2016 - 2020 == ON

- my logic is that each row is a parking ticket so getting the size of each group (count) will suffice for question 3
- sorting descending on max count provides the province with the most parking tickets

In [16]:
sub_df = df[['province', 'date_of_infraction']]

print((sub_df.groupby('province').size()).sort_values(ascending=False))

province
ON    9709298
QC     117938
AB      34816
NY      23800
AZ      20738
       ...   
HI         24
GO         24
NL          6
PR          3
MH          1
Length: 70, dtype: int64


## Date with most tickets in time span == April 1, 2016
- my logic is similar to question 3 answer. Did not find need to use MAX function

In [14]:
print((sub_df.groupby('date_of_infraction').size()).sort_values(ascending=False))

date_of_infraction
20160401    8981
20160901    8714
20161101    8545
20181016    8426
20191210    8206
            ... 
20200518     406
20200413     373
20200412     365
20201225     344
20200410     304
Length: 1827, dtype: int64


## Number of tickets by province for time span

- plotly allows you to filter out ON in order to get a better picture of other provinces ticket activity (ctrl + click ON or any province in plot key)
- visual is interactive

In [28]:
import plotly.express as px

visual_df = sub_df.groupby(['province', 'date_of_infraction'])['province'].count().reset_index(name='ticket_count')

visual_df['date'] = pd.to_datetime(visual_df['date_of_infraction'].astype(str))

fig = px.line(visual_df, x='date', y='ticket_count', color='province', title='Number of Tickets by Province between 2016 - 2020')
fig.show()